In [ ]:
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

In [ ]:
import pm4py

from copy import deepcopy
from pm4py.algo.discovery.dcr_discover import algorithm as alg
from pm4py.objects.dcr import semantics as dcr_semantics

In [ ]:
event_log_file = '/home/vco/Datasets/Sepsis Cases - Event Log.xes'
dcr_title = 'Sepsis Cases'
result_file_prefix = 'sepsis'
sepsis_log = pm4py.read_xes(event_log_file,return_legacy_log_object=True)
sepsis_reference_dcr, sepsis_la = alg.apply(sepsis_log,alg.DCR_BASIC,timed=False)

In [ ]:
def mine_initially_pending(event_log, dcr_model):
    at_least_once_all_traces = set(dcr_model['events'])
    end_excluded_all_traces = set(dcr_model['events'])

    for trace in event_log:
        executed_events = set()
        im = dcr_model['marking']
        dcr = deepcopy(dcr_model)
        complete=True
        for event in trace:
            executed = dcr_semantics.execute(event['concept:name'], dcr)
            if executed:
                executed_events.add(event['concept:name'])
            complete = complete and event['lifecycle:transition']=='complete'
        if complete:
            fm = dcr['marking']
            excluded_events = im['included'].difference(fm['included'])
            at_least_once_all_traces = at_least_once_all_traces.intersection(executed_events)
            end_excluded_all_traces = end_excluded_all_traces.intersection(excluded_events)

    initially_pending = at_least_once_all_traces.union(end_excluded_all_traces)
    return initially_pending

mine_initially_pending(deepcopy(sepsis_log), deepcopy(sepsis_reference_dcr))